In [ ]:
f

In [97]:
## Finding Guido
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import os
from tweepy import Client, Paginator

bearer_token = os.environ.get("TWITTER_BEARER_TOKEN")

client = Client(bearer_token)

In [7]:
user_response = client.get_user(username="gvanrossum", user_fields=["description", "location", "public_metrics"])

In [67]:
gvr = user_response[0].id
gvr

15804774

Who does Guido Van Rossum follow? Originally I made the mistake of using "followers" instead of "following", don't do that! Also, because I find it really confusing to talk about followers and following, I'm going to refer to the accounts that GVR follows as his "subscriptions". GVR subscribes to just over 500 acccounts. Conveniently, the user objects from tweepy map very nicely to pandas dataframes. A little pandas magic with json_normalize lets us explode the "public_metrics" dictionary into new columns in our frame. 

In [104]:
import pandas as pd

def get_subscriptions_df(id):
    """ 
    Function retrieves subscriptions (accounts that a user is following) for a given user id. 
    Function uses pagination if the user account has more than 1000 followers, which is the limit 
    of what the Twitter api will return in one call. 
    """
    subscriptions = []
    for subscription in Paginator(client.get_users_following, id, user_fields=["description", "location", "public_metrics"], max_results=1000):
        subscriptions.extend(subscription.data)
    subscriptions_df = pd.DataFrame(subscriptions)
    
    # public metrics is a dictionary containing tweet and follower counts. 
    # we merge it with the original dataframe. 
    subscriptions_df = subscriptions_df.merge(
        pd.json_normalize(subscriptions_df.public_metrics), left_index=True, right_index=True).\
        drop("public_metrics", axis="columns")
    subscriptions_df["subscriber"] = id
    return subscriptions_df



In [ ]:
gvr_subscriptions_df = get_subscriptions_df(gvr)
gvr_subscriptions_df.head()

We can take a look around at the accounts that Guido is following. They have a very wide range of followers, from 16 to 19,250,000. At the top end, some of the accounts are for celebrities, including: 

- Edward Snowden
- Samantha Power, former US Ambassador to the UN
- Dave Matthews
- Grant Imahara
- Leonard Nimoy

We can also see a lot of organizations, such as Meta, BBC Breaking News, the Mars Rover, etc. It is interesting that so many of these accounts have very small subscription counts of their own, though, much more in line with typical people. Al Gore, for instance, has 2.9 million followers but is subscribing to just 40 accounts himself. 

We are interested in the subscriptions for these accounts because if one of them is following GVR back, it's a pretty crucial indicator that they are part of the python universe. In such a case it would be useful to see their own subscriptions, as it gives us a sense of where they are / what they're interested in in the Python ecosystem. 

We are left with a problem, though. If we tried to get the subscriptions of GVR's subscriptions, we would have 508,982 user records to sort through. There would be a lot of duplication in there for certain, but to get the list itself would be a difficult problem. The twitter api will return a max of 1000 followers per call, and there is a rate limit of 15 calls in 15 minutes. Getting the entire list, then, would take about 5 days! To get under this limit, we will need to take a different approach. We could consider: 

- taking a sample of the followers, knowing that this will leave us with missing nodes in the network. 
- limiting ourselves to 1000 followers max from each follower, which will leave us with nodes and edges missing in our network. 
- testing each account for the likelihood that it is a person rather than an institution or organization; we are more interested in people for this exploration, and perhaps that number is much smaller than the total.
- parsing the user's description for mentions of Python or one of the more popular libraries in the ecosystem (e.g. Django)
- taking an incremental approach, adding people that we "know" to our network and choosing next steps.

I'm kind of curious about the last approach. For example, It would be interesting to see the people who both Wes McKinney (creator of Pandas) and Guido Van Rossum follow. We can search for Wes in the followers list and, sure enough, there he is. Luckily for us he has just under 1000 users so we can retrieve his followers with one call to the API. 

In [63]:
gvr_followers_df[gvr_followers_df.name.str.contains("Wes")]

,description,id,location,name,username,followers_count,following_count,tweet_count,listed_count
337,CTO + co-founder @voltrondata. @ApacheArrow co...,115494880,"Nashville, TN",Wes McKinney,wesmckinn,59052,890,9440,1794
395,"software engineer, Python developer, technical...",8872712,"Silicon Valley, CA, USA",Wesley Chun,wescpy,3982,408,6118,178


In [79]:
wes_followers_df = get_subscriptions_df(115494880)

We would be interested in anyone that both Wes and Guido follow. We don't need network analysis for this one, we can just concatenate their subscriptions and see which accounts are in both. There are only 39 of these! 

In [86]:
follows_both = pd.concat([wes_followers_df, gvr_followers_df]).id.value_counts()
follows_both_ids = follows_both[follows_both > 1].index
len(follows_both_ids)

39

Let's take a closer look at these individuals. As we know already that they're in both sets, we'll look at them in GVR's subscriptions. There are a mix of people and organizations in the list. We are more interested in the people. Of those people, we are more interested, for this analysis, in people who mention python in their bios. We have several directions we could take from here: 
- Jake VanderPlaas, if we were interested in Astronomical / Scientific Python
- Fernando Perez, creator of iPython, if we are interested in the Jupyter community

We might also be interested in the accounts that have the largest number of followers, as it implies some amount of influence in the community, especially if the number of people they are following is realtively low. After exploring a bit I'm interested in Jacob Kaplan-Moss, who was heavily involved with Django. Django is a website framework and one of the more popular uses of Python. 

In [87]:
gvr_followers_df[gvr_followers_df.id.isin(follows_both_ids)]

,description,id,location,name,username,followers_count,following_count,tweet_count,listed_count
13,Symmathecist. Engineering Manager of Developer...,25103,"St. Louis, MO",Jessica Joy Kerr,jessitron,37986,2426,21009,1263
34,"🦑 Co-founder, Dir of Engineering @Cuttlesoft 👩...",863550314,"Denver, CO",Emily Morehouse-Valcarcel,emilyemorehouse,4479,537,1804,119
92,"A free, virtual Python conference being held M...",1616492725,"Nashville, TN",PyTennessee,PyTennessee,1861,38,3643,121
100,"CPython core developer, software architect @Tr...",261665936,"Brisbane, AU (Turrbal land)",Nick Coghlan,ncoghlan_dev,9710,2547,40207,447
102,he/him/his,794723912189775872,NaN,Nathaniel J. Smith,vorpalsmith,2955,210,2466,76
115,,398742438,Paris,Konrad Hinsen (🐘 @khinsen@scholar.social),khinsen,1415,256,18130,96
131,"Coffee and Python, preferably in that order. D...",768197780,Oakland CA,Jake VanderPlas,jakevdp,42680,439,6160,0
158,chef ➡️ developer ➡️ keynote speaker & enginee...,17213939,"🎶 Nashville, TN",adriennefriend,adriennefriend,4793,1135,36506,245
162,"@PyLadiesChicago, past @ThePSF Chair, Director...",21767394,"The Moon, Stars, & Cosmos.","Loooorena ""La 🐯 Tigresa” @ The Cosmos",loooorenanicole,7907,4653,19097,326
165,"Engineering professor, computational scientist...",369758524,"Washington, DC",Lorena Barba,LorenaABarba,9320,1320,12954,402


In [90]:
jacobian_followers_df = get_subscriptions_df(18824526)

14428320    3
17213939    3
16273417    3
14635493    3
24945605    3
           ..
16701253    1
17834877    1
2006841     1
18490730    1
15804774    1
Name: id, Length: 1397, dtype: int64

Having done so, we can now see some accounts that are followed by all 3 individuals. There are 6 accounts in total. One of these is for Jessica McKellar, who I recognize as a major contributor to the Twisted python library. I did a little research and saw that she was also at one point head of the Python Software Foundation. Interestingly, a lot of the individuals on the list have very small subscription accounts. The exception is Nick Coghlan, a CPython developer. There is also an indivdual, Alex Gaynor, who doesn't subscribe to anyone, but to whom many people are subscribed. This is much more the pattern of a media organization than a person, which is most curious. Between all these individuals, there are just over 4500 subscriptions. Let's get 'em all and call it the end of the day for the source of our network. 

In [95]:
follows_three = pd.concat([jacobian_followers_df, gvr_followers_df, wes_followers_df]).id.value_counts()
follows_three = follows_three[follows_three > 2].index
len(follows_three)
jacobian_followers_df[jacobian_followers_df.id.isin(follows_three)]

,description,id,location,name,username,followers_count,following_count,tweet_count,listed_count,subscriber
16,"CTO, https://t.co/FESdoVX9di",24945605,"San Francisco, CA",Jessica McKellar,jessicamckellar,15291,339,1046,521,18824526
19,"Python, software, @coveragepy, typography, jug...",16273417,Boston,Ned Batchelder,nedbat,32186,162,9175,660,18824526
29,chef ➡️ developer ➡️ keynote speaker & enginee...,17213939,"🎶 Nashville, TN",adriennefriend,adriennefriend,4794,1135,36506,245,18824526
34,"CPython core developer, software architect @Tr...",261665936,"Brisbane, AU (Turrbal land)",Nick Coghlan,ncoghlan_dev,9712,2547,40207,447,18824526
45,Python core developer; snarky Canadian,14428320,"Vancouver, British Columbia",Brett Cannon,brettsky,17950,326,16319,574,18824526
49,"software resilience engineer, and generalist t...",14635493,here,Alex Gaynor,alex_gaynor,11730,0,114,769,18824526


In [110]:
followers_frames = []
for subscriber in jacobian_followers_df.query("""following_count > 0""")[jacobian_followers_df.id.isin(follows_three)].id.values:
    followers_frames.append(get_subscriptions_df(subscriber))
followers_df = pd.concat(followers_frames)

/tmp/ipykernel_4714/3086591750.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for subscriber in jacobian_followers_df.query("""following_count > 0""")[jacobian_followers_df.id.isin(follows_three)].id.values:


TooManyRequests: 429 Too Many Requests
Too Many Requests

In [111]:
subscriptions_df = pd.concat([
    gvr_followers_df,
    wes_followers_df,
    jacobian_followers_df,
    followers_df
])
subscriptions_df

,description,id,location,name,username,followers_count,following_count,tweet_count,listed_count,subscriber
0,"I work @Microsoft, making Python more awesome ...",14076724,"Redmond, WA",John Lam,john_lam,4406,586,6182,294,NaN
1,Intl. Relations & MPP Grad 🌍 noob intersection...,187196955,Berlin,Camila Gutiérrez (she/her),Mariacamilagl30,695,444,259,9,NaN
2,"Creator of @datasetteproj, co-creator Django. ...",12497,"San Francisco, CA",Simon Willison,simonw,41285,5171,41075,1595,NaN
3,Partner Software Architect at Microsoft on .NE...,19481808,Redmond,David Fowler 🇧🇧🇺🇸,davidfowl,103735,1319,61329,1607,NaN
4,The #1 Python-focused podcast covering the peo...,3098427092,"Portland, OR USA",Talk Python Podcast,TalkPython,61431,4042,6076,1342,NaN
...,...,...,...,...,...,...,...,...,...,...
321,Working on Python infrastructure at Instagram,17147516,"Seattle, WA",Dino Viehland,DinoViehland,440,134,306,32,14428320.0
322,"software resilience engineer, and generalist t...",14635493,here,Alex Gaynor,alex_gaynor,11730,0,114,769,14428320.0
323,"Engineer, Cyclist, Pythonista, Foodie",38579562,Vancouver,Dr. Doug Latornell,dlatornell,180,442,1416,14,14428320.0
324,,36183197,Idaho,Whitney Cannon,cannonwe,2,0,2,0,14428320.0


There are just under 6000 subscriptions in our fledgling network for about 5200 individuals. Time to bring in networkx!

In [113]:
subscriptions_df.id.nunique()

5208

In [128]:
subscriptions_df.query("""id == 261665936""").subscriber

100     15804774.0
306    115494880.0
34      18824526.0
414     17213939.0
295     14428320.0
Name: subscriber, dtype: float64

In [132]:
import networkx as nx

python_twitterverse = nx.from_pandas_edgelist(subscriptions_df, source="subscriber", target="id", create_using=nx.DiGraph())

I thought it would be nice to bring in the user attributes to associate with each node. I thought that would be as simple as removing our "subscriber" field from the subscriptions dataframe and dropping duplicates.  I had a surprise, though - in between my calls, the public metrics of certain users changed! Looks like Emily Morehouse lost two followers. This highlights the really dynamic nature of Twitter and is a cautionary tale about thinking it is possible to have a truly complete sense of the Python twitterverse, even with infinite computing power and no rate limits. 

In [166]:
users_df = subscriptions_df.drop("subscriber", axis="columns").drop_duplicates().copy()
users_df[users_df.id==users_df[users_df.id.duplicated()].id.values[0]]

,description,id,location,name,username,followers_count,following_count,tweet_count,listed_count
34,"🦑 Co-founder, Dir of Engineering @Cuttlesoft 👩...",863550314,"Denver, CO",Emily Morehouse-Valcarcel,emilyemorehouse,4479,537,1804,119
114,"🦑 Co-founder, Dir of Engineering @Cuttlesoft 👩...",863550314,"Denver, CO",Emily Morehouse-Valcarcel,emilyemorehouse,4477,537,1804,119


We drop duplicates based on the id column only, then set the index for the frame to id. This lets us make a dictionary mapping user ids to their attributes, which is what networkx needs from us. 

In [167]:
users_df = users_df.drop_duplicates(subset=["id"], keep="last")
users_df = users_df.set_index("id")
users_dict = users_df.to_dict(orient="index")

In [169]:
# enhance nodes by adding user details
nx.set_node_attributes(python_twitterverse, users_dict)

In [184]:
users_df["username"].to_dict()

{14076724: 'john_lam',
 187196955: 'Mariacamilagl30',
 19481808: 'davidfowl',
 1456119298892517378: 'margo_seltzer',
 297081267: 'gauravsingh961',
 25663453: 'dsymetweets',
 78942899: 'belmontcapolice',
 1686323288: 'TimSweeneyEpic',
 1326645495624732672: 'ClaudiaRegio',
 2231617016: 'Captain_Joannah',
 1462580222: 'ArmatradingJoan',
 21146468: 'xkcd',
 10915302: 'kashleytwit',
 754854295523037184: 'OndrejCertik',
 1126766656884068352: 'howey_ou',
 373008565: 'davorabbit',
 152862026: 'EdwardTufte',
 301547796: 'MonicaSLam',
 741461784704339968: 'utsav_sha',
 393550165: 'NABA_UNCC',
 324352339: 'andrew_svetlov',
 165840356: 'gjbernat',
 28032548: 'PyOhio',
 237921924: 'caillemillner',
 20144393: 'frgx',
 1936951807: 'gamesbrainiac',
 1960878499: 'aleksamul',
 3241754023: 'SeedsAndBreeds',
 1113304045333098496: 'rosejudge5',
 1118333551630356482: 'amphibianne',
 193059794: 'JoanneHastieArt',
 139199211: 'schnarfed',
 15431437: 'saschel',
 9929072: 'erdmann',
 387878582: 'space_quakes',


In [188]:
python_twitterverse = nx.relabel_nodes(python_twitterverse, users_df["username"].to_dict())

In [189]:
nx.write_graphml(python_twitterverse, "python_twitterverse.graphml")

In [190]:
for clique in nx.find_cliques(python_twitterverse.to_undirected()):
    if len(clique) > 6:
        print(clique)

['ncoghlan_dev', 'adriennefriend', 'gvanrossum', 'jessicamckellar', 'nedbat', 'wesmckinn', 'jacobian']
['ncoghlan_dev', 'adriennefriend', 'gvanrossum', 'jessicamckellar', 'nedbat', 'wesmckinn', 'ThePSF']
['ncoghlan_dev', 'adriennefriend', 'gvanrossum', 'jessicamckellar', 'nedbat', 'brandon_rhodes', 'jacobian']
['ncoghlan_dev', 'brettsky', 'gvanrossum', 'wesmckinn', 'nedbat', 'jacobian', 'alex_gaynor']
['ncoghlan_dev', 'brettsky', 'gvanrossum', 'wesmckinn', 'nedbat', 'jacobian', 'jessicamckellar']
['ncoghlan_dev', 'brettsky', 'gvanrossum', 'wesmckinn', 'nedbat', 'ThePSF', 'jessicamckellar']
['ncoghlan_dev', 'brettsky', 'gvanrossum', 'brandon_rhodes', 'jacobian', 'jessicamckellar', 'nedbat']
